In [ ]:
import sympy as sp
x,y,z = sp.symbols('x y z')
def devgrad(f):
    Gf = sp.Matrix([[f[0].diff(x),f[0].diff(y),f[0].diff(z)],
                    [f[1].diff(x),f[1].diff(y),f[1].diff(z)],
                    [f[2].diff(x),f[2].diff(y),f[2].diff(z)]])
    return Gf - Gf.trace()/3*sp.Matrix.eye(3)
def curl(f):
    Cf = sp.Matrix([[f[0,2].diff(y)-f[0,1].diff(z),f[0,0].diff(z)-f[0,2].diff(x),f[0,1].diff(x)-f[0,0].diff(y)],
                    [f[1,2].diff(y)-f[1,1].diff(z),f[1,0].diff(z)-f[1,2].diff(x),f[1,1].diff(x)-f[1,0].diff(y)],
                    [f[2,2].diff(y)-f[2,1].diff(z),f[2,0].diff(z)-f[2,2].diff(x),f[2,1].diff(x)-f[2,0].diff(y)]])
    return Cf
def symcurl(f):
    Cf = curl(f)
    return (Cf+Cf.transpose())/2
def div(f):
    return sp.Matrix([f[0,0].diff(x)+f[0,1].diff(y)+f[0,2].diff(z), 
                      f[1,0].diff(x)+f[1,1].diff(y)+f[1,2].diff(z),
                      f[2,0].diff(x)+f[2,1].diff(y)+f[2,2].diff(z)])
def divdiv(f):
    return (f[0,0].diff(x).diff(x) +
            f[1,0].diff(y).diff(x) +
            f[2,0].diff(z).diff(x) +
            f[0,1].diff(x).diff(y) +
            f[1,1].diff(y).diff(y) +
            f[2,1].diff(z).diff(y) +
            f[0,2].diff(x).diff(z) +
            f[1,2].diff(y).diff(z) +
            f[2,2].diff(z).diff(z) )
def simplifyMat(A):
    for i in range(A.rows):
        for j in range(A.cols):
            A[i,j] = A[i,j].simplify()
    return A
def ev(f,ax,ay,az):
    return f.subs(x,ax).subs(y,ay).subs(z,az)
def S(A):
    SA = A.transpose() - A.trace()*sp.Matrix.eye(3)
    return simplifyMat(SA)
def dt(A):
    At = sp.Matrix.eye(3)
    for i in range(A.rows):
        for j in range(A.cols):
            At[i,j] = A[i,j].diff(t)
    return At

In [ ]:
import re
def cppcode(f):
    s = sp.ccode(f.simplify())
    s = re.sub("sin","autodiff::detail::sin",s)
    s = re.sub("cos","autodiff::detail::cos",s)
    s = re.sub("exp","autodiff::detail::exp",s)
    s = re.sub("pow","autodiff::detail::pow",s)
    return s
ADB = "      return autodiff1st(\n        [](autodiff::real x, autodiff::real y, autodiff::real z)->autodiff::real {\n          return "
ADE = ";\n        },X,dx,dy,dz);\n"
def ADPrint(A):
    if (A.cols == 1):
        print("    if (i == 0) {\n",ADB,cppcode(A[0]),ADE,
              "    } else if (i == 1) {\n",ADB,cppcode(A[1]),ADE,
              "    } else {\n",ADB,cppcode(A[2]),ADE,
              "    };",sep='')
    else:
        print("    if (i == 0 && j == 0) {\n",ADB,cppcode(A[0,0]),ADE,
              "    } else if (i == 0 && j == 1) {\n",ADB,cppcode(A[0,1]),ADE,
              "    } else if (i == 0 && j == 2) {\n",ADB,cppcode(A[0,2]),ADE,
              "    } else if (i == 1 && j == 0) {\n",ADB,cppcode(A[1,0]),ADE,
              "    } else if (i == 1 && j == 1) {\n",ADB,cppcode(A[1,1]),ADE,
              "    } else if (i == 1 && j == 2) {\n",ADB,cppcode(A[1,2]),ADE,
              "    } else if (i == 2 && j == 0) {\n",ADB,cppcode(A[2,0]),ADE,
              "    } else if (i == 2 && j == 1) {\n",ADB,cppcode(A[2,1]),ADE,
              "    } else {\n",ADB,cppcode(A[2,2]),ADE,
              "    };",sep='')


In [ ]:
def print0(f):
    df = devgrad(f)
    print("  static double f(unsigned i, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(f)
    print("  }\n  static double df(unsigned i, unsigned j, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(df)
    print("  }")
def print1(f):
    df = symcurl(f)
    print("  static double f(unsigned i, unsigned j, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(f)
    print("  }\n  static double df(unsigned i, unsigned j, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(df)
    print("  }")
def print2(f):
    df = divdiv(f)
    print("  static double f(unsigned i, unsigned j, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(f)
    print("  }\n  static double df(const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    print(ADB,cppcode(df),ADE,";",sep='')
    print("  }")

In [ ]:
print0(sp.Matrix([[(3*x**3+2*x**2-x+1)*(y**2+3*y-1)*(2*z**2-z+2)],
                  [(5*x**2-x+1)*(4*y**3-y**2+2*y-1)*(3*z**2-z+3)],
                  [(1*x**2-x+4)*(-y**2+3*y-1)*(-z**3-2*z**2-z+4)]]))

In [ ]:
print1(sp.Matrix([[(2*x**2-x+1)*(y**2+3*y-1)*(2*z**2-z+2),(x**3-2*x**2-x+1)*(3*y-1)*(2*z**2-z+2),(2*x**3+x**2-x+1)*(y**2+3*y-1)*(z+2)],
                  [(x+1)*(4*y**3-y**2+2*y-1)*(3*z**2-z+3),(3*x**2-x+1)*(-y**2+1*y-1)*(3*z**2-z+3),(5*x**2-x+1)*(4*y**3-y**2+2*y-1)*(z-1)],
                  [(-x+4)*(-y**2+3*y-1)*(-z**3-2*z**2-z+3),(2*x**2-x+4)*(y-1)*(-z**3-2*z**2-z+4),-(2*x**2-x+1)*(y**2+3*y-1)*(2*z**2-z+2)-(3*x**2-x+1)*(-y**2+1*y-1)*(3*z**2-z+3)]]))


In [ ]:
print2(sp.Matrix([[(3*x**3+2*x**2-x+1)*(3*y-1)*(z+2),(2*x**2-x+1)*(y**2+3*y-1)*(-z+2),(x**2-x+1)*(y+1)*(2*z**2-z+2)],
                  [(2*x**2-x+1)*(y**2+3*y-1)*(-z+2),(x+1)*(4*y**3-y**2+2*y-1)*(z+1),(-x+4)*(-y**2+3*y-1)*(-2*z**2-z+4)],
                  [(x**2-x+1)*(y+1)*(2*z**2-z+2),(-x+4)*(-y**2+3*y-1)*(-2*z**2-z+4),(x+4)*(y-1)*(-z**3-2*z**2-z+4)]]))
